In [ ]:
#Iniciar servidor Local web2py.py
python web2py.py -a abc

#Acessar no navegador
http://127.0.0.1:8000

http://127.0.0.1:8000/gestao_suprimentos

## desligar to shutdown the web2py server
"taskkill /f /pid 1940" 


In [ ]:
# Pseudo-código de atualização de estoque para movimentação de entrada
def registrar_entrada(item_id, quantidade, local_estocado):
    # Consulta o saldo atual
    estoque_atual = consultar_estoque(item_id, local_estocado)
    
    if estoque_atual:  # Se o item já estiver no estoque
        novo_saldo = estoque_atual + quantidade
        atualizar_estoque(item_id, local_estocado, novo_saldo)
    else:  # Se o item não estiver no estoque, insere um novo registro
        inserir_novo_estoque(item_id, local_estocado, quantidade)

# Pseudo-código de atualização de estoque para movimentação de saída
def registrar_saida(item_id, quantidade, local_estocado):
    # Consulta o saldo atual
    estoque_atual = consultar_estoque(item_id, local_estocado)
    
    if estoque_atual and estoque_atual >= quantidade:  # Verifica se há saldo suficiente
        novo_saldo = estoque_atual - quantidade
        atualizar_estoque(item_id, local_estocado, novo_saldo)
    else:
        raise Exception("Saldo insuficiente no estoque")


In [ ]:
def cadastrar_usuario():
    """
    Função para cadastrar novos usuários no sistema.
    """
    # Define o formulário baseado na tabela ac_usuarios
    form = SQLFORM(db.ac_usuarios)

    # Validação do formulário
    if form.process().accepted:
        response.flash = 'Usuário cadastrado com sucesso!'
        redirect(URL('listar_usuarios'))  # Redireciona após o cadastro bem-sucedido
    elif form.errors:
        response.flash = 'O formulário contém erros, por favor verifique.'

    # Renderiza a página com o formulário
    return dict(form=form)


In [ ]:
db = DAL('mysql://root:@localhost/gestao_suprimentos', migrate_enabled=True)

db.define_table('ac_usuarios',
    Field('nome', 'string', label="Nome", length=100),
    Field('nivel_acesso', 'string', requires=IS_IN_SET(['Administrador', 'Usuário Padrão', 'Gestor']), label="Nível de Acesso"),
    Field('login', 'string', length=50, unique=True, label="Login"),
    Field('senha', 'password', label="Senha"),
    Field('status', 'string', requires=IS_IN_SET(['Ativo', 'Inativo']), label="Status"),
    Field('limite_aprovacao', 'double', label="Limite de Aprovação"),
    Field('centro_custo', 'string', length=50, label="Centro de Custo"),
    migrate=False
)

db.define_table('ac_item',
    Field('descricao_item', 'string', length=255, label="Descrição do Item"),
    Field('valor_unitario', 'double', label="Valor Unitário"),
    Field('unidade_medida', 'string', length=50, label="Unidade de Medida"),
    Field('parametros', 'string', length=255, label="Parâmetros"),
    # Inclua qualquer outro campo relevante para itens/produtos/serviços
    migrate=False
)

db.define_table('ac_estoque',
    Field('item_id', 'reference ac_item', label="Item"),
    Field('local_estocado', 'string', length=60, label="Local Estocado"),
    Field('quantidade_atual', 'integer', label="Quantidade Atual"),
    migrate=False
)

db.define_table('ac_movimentacoes_estoque',
    Field('item_id', 'reference ac_item', label="Item"),
    Field('usuario_id', 'reference ac_usuarios', label="Usuário"),
    Field('tipo_movimentacao', 'string', requires=IS_IN_SET(['entrada', 'saida']), label="Tipo de Movimentação"),
    Field('quantidade', 'integer', label="Quantidade"),
    Field('data_movimentacao', 'date', label="Data da Movimentação"),
    Field('local_estocado', 'string', length=60, label="Local Estocado"),
    Field('valor_movimentacao', 'double', label="Valor da Movimentação"),
    migrate=False
)


# -------------------------------------------------------------------------
# after defining tables, uncomment below to enable auditing
# -------------------------------------------------------------------------
auth.enable_record_versioning(db)


In [ ]:
{{extend 'layout.html'}}

{{block header}}
<div class="jumbotron jumbotron-fluid background" style="background-color: #333; color:white; padding:30px;word-wrap:break-word;">
  <h3 class="mb-4 text-center">Cadastrar Usuarios</h3>
</div>
{{end}}

<div class="row">
    <!-- <h1>Cadastrar Usuário</h1> -->
    {{=form}}
</div>